In [ ]:
from collections import Counter

class Hand:
    def __init__(self, cards, points) -> None:
        self.cards = cards
        self.points = points
        self.prev_hand = None
        self.next_hand = None


def sort_hands(hands):
    base_hand = hands[0]
    trans = {
        "A": 14,
        "K": 13,
        "Q": 12,
        "J": 1,
        "T": 10,
        "9": 9,
        "8": 8,
        "7": 7,
        "6": 6,
        "5": 5,
        "4": 4,
        "3": 3,
        "2": 2
    }
    for h in hands[1:]:
        prev_state = None
        card_pos = 0
        cycle_cnt = 0
        while(1):
            cycle_cnt += 1
            if cycle_cnt >= 1000:

                raise ValueError
            if base_hand == h:

                break
            while trans[h.cards[card_pos]] == trans[base_hand.cards[card_pos]]:
                card_pos += 1
                if card_pos >= len(h.cards):
                    card_pos = 0
            if trans[h.cards[card_pos]] > trans[base_hand.cards[card_pos]]:
                if base_hand.prev_hand is not None:
                    if prev_state == "NEXT":
                        h.prev_hand = base_hand.prev_hand
                        h.next_hand = base_hand
                        base_hand.prev_hand.next_hand = h
                        base_hand.prev_hand = h
                        base_hand = h
                    else:

                        prev_state = "PREV"
                        base_hand = base_hand.prev_hand
                        card_pos = 0
                else:
                    base_hand.prev_hand = h
                    h.next_hand = base_hand
            elif trans[h.cards[card_pos]] < trans[base_hand.cards[card_pos]]:
                if base_hand.next_hand is not None:
                    if prev_state == "PREV":
                        h.next_hand = base_hand.next_hand
                        h.prev_hand = base_hand
                        base_hand.next_hand.prev_hand = h
                        base_hand.next_hand = h
                        base_hand = h
                    else:

                        prev_state = "NEXT"
                        base_hand = base_hand.next_hand
                        card_pos = 0
                else:
                    base_hand.next_hand = h
                    h.prev_hand = base_hand
    while h.prev_hand is not None:
        h = h.prev_hand
    return h


def get_raw_score(hands):
    raw_scores = dict()
    ranks = {
        5: 7,
        4: 6,
        (3, 2): 5,
        3: 4,
        (2, 2): 3,
        2: 2,
    }
    total = 0
    for h in hands:
        total += 1
        c = Counter(h.cards)
        j = 0
        if "J" in c:
            j = c["J"]
            del c["J"]
        r = list(c.values())
        r.sort()
        if not r:
            r = [0]
        r[-1] = r[-1] + j
        if len(r) >= 2 and (r[-1], r[-2]) in ranks:
            if ranks[(r[-1], r[-2])] not in raw_scores:
                raw_scores[ranks[(r[-1], r[-2])]] = list()
            raw_scores[ranks[(r[-1], r[-2])]].append(h)
        elif (r[-1]) in ranks:
            if ranks[r[-1]] not in raw_scores:
                raw_scores[ranks[r[-1]]] = list()
            raw_scores[ranks[r[-1]]].append(h)
        else:
            if 1 not in raw_scores:
                raw_scores[1] = list()
            raw_scores[1].append(h)

    return raw_scores, total


input = open('input').readlines()

hands = list()
for line in input:
    cards, score = line.strip().split(' ')
    hands.append(Hand(cards, int(score)))
d, hand_cnt = get_raw_score(hands)
total = 0
for i in range(7,0, -1):
    if i in d:
        if len(d[i]) == 1:
            total += d[i][0].points * hand_cnt
            hand_cnt -= 1
        else:
            h = sort_hands(d[i])
            while h is not None:
                total += h.points * hand_cnt
                h = h.next_hand
                hand_cnt -= 1
print(total)